# Feature Selection for log_diff_model (Multivariate Model)

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from bdi_predict.ml_logic.sequence_gen import WindowGenerator
from tensorflow.keras import layers
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
bdi = pd.read_csv("../raw_data/data/BDI/log_diff_BDI_daily.csv")
bdi.head(2)

Unnamed: 0        Date   Price  abs_price  log_price  log_diff
0           0  1995-01-03  1964.0        NaN   3.293141       NaN
1           1  1995-01-04  1961.0       -3.0   3.292478 -0.000664

In [3]:
bdi["Date"] = pd.to_datetime(bdi["Date"])
bdi.set_index("Date", inplace=True)
bdi= bdi[bdi.index != "1995-01-03"]
bdi.drop(columns="Unnamed: 0", inplace=True)
bdi.head(2)

Price  abs_price  log_price  log_diff
Date                                              
1995-01-04  1961.0       -3.0   3.292478 -0.000664
1995-01-05  1967.0        6.0   3.293804  0.001327

In [ ]:
pd.read_csv("")

In [57]:
feat = pd.read_csv("../raw_data/data/merged_features.csv")
feat["Date"] = pd.to_datetime(feat["Date"])
feat.set_index("Date", inplace=True)
feat.drop(columns="Unnamed: 0", inplace=True)
feat = feat[["close_y", "Nickel_x"]]
feat.rename(columns={"close_y":"iron", 
                    "Nickel_x":"nickel"}, inplace=True)
comb=feat
comb

iron        nickel
Date                               
1995-01-03  12.270000   8505.450000
1995-01-04  12.270000   8505.450000
1995-01-05  12.270000   8505.450000
1995-01-06  12.270000   8505.450000
1995-01-09  12.270000   8505.450000
...               ...           ...
2022-11-08  27.540690  10264.567966
2022-11-09  26.013621  10003.955424
2022-11-10  24.486552   9743.342881
2022-11-11  22.959483   9482.730339
2022-11-14  21.432414   9222.117797

[7365 rows x 2 columns]

In [21]:
df = pd.merge(bdi, comb, on="Date", how="inner")
#df.drop(columns="Price_y", inplace=True)


In [22]:
df.head(3)

Price  abs_price  log_price  log_diff     DAP   iron   nickel
Date                                                                      
1995-01-04  1961.0       -3.0   3.292478 -0.000664  198.63  12.27  8505.45
1995-01-05  1967.0        6.0   3.293804  0.001327  198.63  12.27  8505.45
1995-01-06  1983.0       16.0   3.297323  0.003518  198.63  12.27  8505.45

# Data Cleaning

## Holdout Method (manual, chronological)

In [23]:
input_length = len(df)

In [24]:
def train_test_split(df:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> tuple:
    '''
    Returns a train dataframe and a test dataframe (df_train, df_test)
    from which one can sample (X,y) sequences using TimeseriesGenerator.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    
    # TRAIN SET

    last_train_idx = round(train_test_ratio * len(df))
    df_train = df.iloc[0:last_train_idx, :]

    # TEST SET
 
    first_test_idx = last_train_idx - input_length
    df_test = df.iloc[first_test_idx:, :]

    return (df_train, df_test)

In [25]:
df_train, df_test = train_test_split(df=df, train_test_ratio=0.8, input_length=input_length)

In [26]:
X_train = df_train[["Price","DAP", "nickel", "iron"]]
y_train = df_train["log_diff"]

X_test  = df_test[["Price","DAP", "nickel", "iron"]]
y_test = df_test["log_diff"]

In [27]:
#Feature Scaling

scaler_X = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.fit_transform(X_test)



In [44]:
generator = TimeseriesGenerator(X_train_scaled, y_train, length=20, batch_size=10, sampling_rate=1, stride=1)

In [45]:
len(generator)

564

In [46]:
generator_val = TimeseriesGenerator(X_test_scaled, y_test, length=20, batch_size=10, sampling_rate=1, stride=1)

In [47]:
for X, y in generator:
    print(X.shape, y.shape)
    print(X[0], y[0])
    #listy = [float(X[0][i]) for i in range(len(X[0]))]
    #print(listy)
    break

(10, 20, 4) (10,)
[[0.14519214 0.06948717 0.09591795 0.00194009]
 [0.14571379 0.06948717 0.09591795 0.00194009]
 [0.14710485 0.06948717 0.09591795 0.00194009]
 [0.14806121 0.06948717 0.09591795 0.00194009]
 [0.14988698 0.06948717 0.09591795 0.00194009]
 [0.15023474 0.06948717 0.09591795 0.00194009]
 [0.15188663 0.06948717 0.09591795 0.00194009]
 [0.15379934 0.06948717 0.09591795 0.00194009]
 [0.15440793 0.06948717 0.09591795 0.00194009]
 [0.15475569 0.06948717 0.09591795 0.00194009]
 [0.15475569 0.06948717 0.09591795 0.00194009]
 [0.15466875 0.06948717 0.09591795 0.00194009]
 [0.1537124  0.06948717 0.09591795 0.00194009]
 [0.15240828 0.06948717 0.09591795 0.00194009]
 [0.15136498 0.06948717 0.09591795 0.00194009]
 [0.1501478  0.06948717 0.09591795 0.00194009]
 [0.14823509 0.06948717 0.09591795 0.00194009]
 [0.14710485 0.06948717 0.09591795 0.00194009]
 [0.1465832  0.06948717 0.09591795 0.00194009]
 [0.14623544 0.06948717 0.09591795 0.00194009]] -0.0015435717560743


# The Model

In [48]:
# Learning Rate Scheduler: 

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)

In [49]:
#Optimizers

rmsprop = RMSprop(learning_rate=lr_schedule) #or 0.001 for inital test
adam = Adam(learning_rate=lr_schedule)

In [50]:
#Early Stopping Criterion:

es = EarlyStopping(monitor="val_mae", patience=100, restore_best_weights=True)

In [51]:
#Initializing and compiling model:

def init_model():
    
    model = Sequential()

    #LSTM layers
    model.add(layers.LSTM(60, activation="tanh", input_shape=(20,4), return_sequences=False))

    #Dense layers
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(1, activation="linear"))

    #compiling model
    model.compile(loss="mse", optimizer=rmsprop, metrics="mae")
    
    return model

# Training and Testing

In [52]:
#Instantiating model

model = init_model()

In [53]:
#Model layers & params overview:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 60)                15600     
                                                                 
 dense_2 (Dense)             (None, 100)               6100      
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21,801
Trainable params: 21,801
Non-trainable params: 0
_________________________________________________________________


In [54]:
#TRAINING THE MODEL:

history = model.fit(
    generator, epochs=1000, validation_data=generator_val, shuffle=True, callbacks=es
)

Epoch 1/1000
564/564 [==============================] - 3s 3ms/step - loss: 2.5831e-04 - mae: 0.0104 - val_loss: 1.8467e-04 - val_mae: 0.0108
Epoch 2/1000
564/564 [==============================] - 1s 2ms/step - loss: 9.1589e-05 - mae: 0.0069 - val_loss: 1.2996e-04 - val_mae: 0.0087
Epoch 3/1000
564/564 [==============================] - 1s 2ms/step - loss: 7.0533e-05 - mae: 0.0059 - val_loss: 1.0760e-04 - val_mae: 0.0078
Epoch 4/1000
564/564 [==============================] - 1s 2ms/step - loss: 6.3402e-05 - mae: 0.0055 - val_loss: 1.0351e-04 - val_mae: 0.0077
Epoch 5/1000
564/564 [==============================] - 1s 2ms/step - loss: 6.1241e-05 - mae: 0.0055 - val_loss: 1.0681e-04 - val_mae: 0.0078
Epoch 6/1000
564/564 [==============================] - 1s 2ms/step - loss: 5.9735e-05 - mae: 0.0054 - val_loss: 1.0245e-04 - val_mae: 0.0076
Epoch 7/1000
564/564 [==============================] - 2s 3ms/step - loss: 5.9281e-05 - mae: 0.0052 - val_loss: 1.1101e-04 - val_mae: 0.0081
Epoch 

564/564 [==============================] - 1s 3ms/step - loss: 4.8706e-05 - mae: 0.0048 - val_loss: 8.9265e-05 - val_mae: 0.0072
Epoch 59/1000
564/564 [==============================] - 1s 3ms/step - loss: 4.9135e-05 - mae: 0.0049 - val_loss: 9.5451e-05 - val_mae: 0.0076
Epoch 60/1000
564/564 [==============================] - 1s 3ms/step - loss: 4.9430e-05 - mae: 0.0048 - val_loss: 1.1429e-04 - val_mae: 0.0086
Epoch 61/1000
564/564 [==============================] - 1s 3ms/step - loss: 4.8381e-05 - mae: 0.0048 - val_loss: 9.6004e-05 - val_mae: 0.0078
Epoch 62/1000
564/564 [==============================] - 1s 3ms/step - loss: 4.9240e-05 - mae: 0.0048 - val_loss: 8.8599e-05 - val_mae: 0.0071
Epoch 63/1000
564/564 [==============================] - 1s 3ms/step - loss: 4.8832e-05 - mae: 0.0048 - val_loss: 9.5996e-05 - val_mae: 0.0078
Epoch 64/1000
564/564 [==============================] - 1s 3ms/step - loss: 4.7884e-05 - mae: 0.0048 - val_loss: 8.5191e-05 - val_mae: 0.0070
Epoch 65/1000

564/564 [==============================] - 1s 3ms/step - loss: 2.3830e-05 - mae: 0.0032 - val_loss: 4.4420e-05 - val_mae: 0.0045
Epoch 116/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.3566e-05 - mae: 0.0032 - val_loss: 5.7809e-05 - val_mae: 0.0056
Epoch 117/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.3088e-05 - mae: 0.0032 - val_loss: 3.9156e-05 - val_mae: 0.0041
Epoch 118/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.2755e-05 - mae: 0.0032 - val_loss: 6.0263e-05 - val_mae: 0.0057
Epoch 119/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.3044e-05 - mae: 0.0032 - val_loss: 4.5397e-05 - val_mae: 0.0046
Epoch 120/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.2939e-05 - mae: 0.0032 - val_loss: 4.0523e-05 - val_mae: 0.0042
Epoch 121/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.3206e-05 - mae: 0.0032 - val_loss: 6.0080e-05 - val_mae: 0.0057
Epoch 1

Epoch 172/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0751e-05 - mae: 0.0029 - val_loss: 5.9486e-05 - val_mae: 0.0055
Epoch 173/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0795e-05 - mae: 0.0029 - val_loss: 4.1937e-05 - val_mae: 0.0043
Epoch 174/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0126e-05 - mae: 0.0029 - val_loss: 4.6925e-05 - val_mae: 0.0048
Epoch 175/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0630e-05 - mae: 0.0029 - val_loss: 4.6838e-05 - val_mae: 0.0048
Epoch 176/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0266e-05 - mae: 0.0029 - val_loss: 4.9230e-05 - val_mae: 0.0048
Epoch 177/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0231e-05 - mae: 0.0029 - val_loss: 4.1013e-05 - val_mae: 0.0044
Epoch 178/1000
564/564 [==============================] - 1s 3ms/step - loss: 2.0264e-05 - mae: 0.0028 - val_loss: 5.1707e-05 - val_mae:

Epoch 229/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.8758e-05 - mae: 0.0027 - val_loss: 6.4757e-05 - val_mae: 0.0055
Epoch 230/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.8814e-05 - mae: 0.0027 - val_loss: 4.6745e-05 - val_mae: 0.0046
Epoch 231/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.8636e-05 - mae: 0.0027 - val_loss: 4.6837e-05 - val_mae: 0.0046
Epoch 232/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.9188e-05 - mae: 0.0028 - val_loss: 7.7274e-05 - val_mae: 0.0066
Epoch 233/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.8994e-05 - mae: 0.0028 - val_loss: 8.8275e-05 - val_mae: 0.0063
Epoch 234/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.8820e-05 - mae: 0.0027 - val_loss: 5.6467e-05 - val_mae: 0.0051
Epoch 235/1000
564/564 [==============================] - 1s 2ms/step - loss: 1.8942e-05 - mae: 0.0027 - val_loss: 6.0759e-05 - val_mae:

In [55]:
model.evaluate(generator_val)

140/140 [==============================] - 0s 764us/step - loss: 3.7506e-05 - mae: 0.0039


[3.750615724129602e-05, 0.003944895230233669]

In [56]:
# Learning Curves:
plot_history(history)

array([<AxesSubplot: title={'center': 'MSE'}, xlabel='Epoch', ylabel='Loss'>,
       <AxesSubplot: title={'center': 'MAE'}, xlabel='Epoch', ylabel='MAE'>],
      dtype=object)

In [98]:
#Baseline MAE
0.0048812746058099115

0.0048812746058099115

# Learning Curves viz code:

In [40]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # Loss:MSE
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # Metrics:MAE
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax